In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import re

In [2]:
#드라이브에 접근할 수 있도록 아래 코드 입력
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
chart_1904 = '/content/drive/My Drive/bc카드/주식일간기본지표_201904.csv'
chart_1905 = '/content/drive/My Drive/bc카드/주식일간기본지표_201905.csv'

In [4]:
chart_1904 = pd.read_csv(chart_1904, encoding="euc-kr")
chart_1904.head()
chart_1904.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   기준일자          20000 non-null  int64  
 1   주식종목값         20000 non-null  object 
 2   주식종목명         20000 non-null  object 
 3   주가순자산비율       19977 non-null  float64
 4   주가수익비율        19977 non-null  float64
 5   주가현금흐름비율      19977 non-null  float64
 6   주가매출비율        19977 non-null  float64
 7   매출액대비기업가치비율   19978 non-null  float64
 8   세전이익대비기업가치비율  19978 non-null  float64
 9   실배당수익율        12641 non-null  float64
 10  우편번호          19977 non-null  float64
 11  있음,없음         7396 non-null   object 
dtypes: float64(8), int64(1), object(3)
memory usage: 1.8+ MB


In [5]:
chart_1905 = pd.read_csv(chart_1905, encoding="euc-kr")
chart_1905.head()
chart_1905.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   기준일자          20000 non-null  int64  
 1   주식종목값         20000 non-null  object 
 2   주식종목명         20000 non-null  object 
 3   주가순자산비율       19979 non-null  float64
 4   주가수익비율        19979 non-null  float64
 5   주가현금흐름비율      19979 non-null  float64
 6   주가매출비율        19979 non-null  float64
 7   매출액대비기업가치비율   19979 non-null  float64
 8   세전이익대비기업가치비율  19979 non-null  float64
 9   실배당수익율        12512 non-null  float64
 10  우편번호          19979 non-null  float64
 11  있음,없음         7539 non-null   object 
dtypes: float64(8), int64(1), object(3)
memory usage: 1.8+ MB


In [6]:
chart_2019 = pd.concat([chart_1904,chart_1905])

In [7]:
chart_2019.head()

,기준일자,주식종목값,주식종목명,주가순자산비율,주가수익비율,주가현금흐름비율,주가매출비율,매출액대비기업가치비율,세전이익대비기업가치비율,실배당수익율,우편번호,"있음,없음"
0,20190416,A950180,SNK,NaN,NaN,NaN,NaN,-0.99,-1.82,NaN,NaN,NaN
1,20190412,A000020,동화약품,0.90,26.63,-61.50,0.87,0.54,9.96,0.0,4522.0,NaN
2,20190409,A000020,동화약품,0.88,26.05,-60.15,0.86,0.52,9.60,0.0,4522.0,NaN
3,20190402,A000020,동화약품,0.88,26.05,-60.15,0.86,0.52,9.60,0.0,4522.0,NaN
4,20190429,A000020,동화약품,0.93,27.33,-63.10,0.90,0.56,10.38,0.0,4522.0,NaN


In [8]:
issue = '/content/drive/My Drive/bc카드/이슈지표.csv'

In [9]:
issue_data = pd.read_csv(issue, encoding="euc-kr")
issue_data.head()

,기준일자,주식종목값,주식종목명,이슈내용,이슈분류이용문서수,우편번호,적재일시,센터코드
0,20190424,51630,진양화학,"[{'words': ['화학', '선', '차트', '진양', '상승', '주가',...",5,44781.0,2.020000e+13,BBP14
1,20190423,82920,비츠로셀,"[{'words': ['계약', '경제', '비츠', '규모', '전지', '언론사...",5,31816.0,2.020000e+13,BBP14
2,20190417,23890,한국아트라스비엑스,"[{'words': ['라스', '아트', 'BX', '주주', '밸류', '너스'...",5,34365.0,2.020000e+13,BBP14
3,20190419,97800,윈팩,"[{'words': ['경제', '서울', '윈', '팩', '헤럴드', '언론사'...",5,17172.0,2.020000e+13,BBP14
4,20190417,4960,한신공영,"[{'words': ['한신공영', '봉사', '기자', '라오스', '가구', '...",5,17172.0,2.020000e+13,BBP14


In [10]:
from collections import OrderedDict, defaultdict

In [12]:
issue_data.이슈내용.to_dict()

{0: "[{'words': ['화학', '선', '차트', '진양', '상승', '주가', '경제', '단기', '중기', '한국'], 'wordweights': [0.019678357988595963, 0.01878388226032257, 0.01699492707848549, 0.016100451350212097, 0.01520597841590643, 0.01520597841590643, 0.014311502687633038, 0.014311502687633038, 0.014311502687633038, 0.014311500824987888]}]",
 1: "[{'words': ['계약', '경제', '비츠', '규모', '전지', '언론사', '해당', '미터', '뉴스', '리튬'], 'wordweights': [0.02117687091231346, 0.02000037021934986, 0.014117876067757607, 0.01294137816876173, 0.01294137816876173, 0.01294137816876173, 0.011764878407120705, 0.011764878407120705, 0.010588381439447403, 0.010588380508124828]}]",
 2: "[{'words': ['라스', '아트', 'BX', '주주', '밸류', '너스', '파트', '있', '최근', '한국'], 'wordweights': [0.017554210498929024, 0.017554208636283875, 0.017005637288093567, 0.0098741939291358, 0.0098741939291358, 0.009325620718300343, 0.009325619786977768, 0.008228476159274578, 0.008228476159274578, 0.007131330668926239]}]",
 3: "[{'words': ['경제', '서울', '윈', '팩', '헤럴드', '언론사', '투자', '